# Power Plants in Germany

This file covers german power plants. It downloads the power plant list from the BNetzA and augments it with more information.

## Table of Contents
* [Power Plants in Germany](#Power-Plants-in-Germany)
* [License](#License)
* [Prepare the environment](#Prepare-the-environment)
* [Specify the source URLs:](#Specify-the-source-URLs:)
* [Define functions](#Define-functions)
* [Downloads](#Downloads)
	* [Download the BNetzA power plant list](#Download-the-BNetzA-power-plant-list)
	* [Download the Uba Plant list](#Download-the-Uba-Plant-list)
* [Translate contents](#Translate-contents)
	* [Columns](#Columns)
	* [Fuel types](#Fuel-types)
	* [Power plant status](#Power-plant-status)
	* [CHP Capability](#CHP-Capability)
	* [UBA Columns](#UBA-Columns)
	* [UBA Technologies](#UBA-Technologies)
* [Process data](#Process-data)
	* [Set index to the BNetzA power plant ID](#Set-index-to-the-BNetzA-power-plant-ID)
	* [Merge data from UBA List](#Merge-data-from-UBA-List)
		* [case 1-1](#case-1-1)
		* [case n-1](#case-n-1)
		* [case 1-n](#case-1-n)
		* [Merge into plantlist](#Merge-into-plantlist)
	* [Delete fuels not in focus](#Delete-fuels-not-in-focus)
	* [Convert input colums to usable data types](#Convert-input-colums-to-usable-data-types)
	* [Add efficiency data](#Add-efficiency-data)
		* [Efficiencies from research](#Efficiencies-from-research)
			* [Plot efficiencies by year of commissioning](#Plot-efficiencies-by-year-of-commissioning)
			* [Determine least-squares approximation based on researched data](#Determine-least-squares-approximation-based-on-researched-data)
			* [Apply efficiency approximation from least squares approximation](#Apply-efficiency-approximation-from-least-squares-approximation)
		* [Efficiencies from literature](#Efficiencies-from-literature)
			* [Import data](#Import-data)
			* [Apply efficiency approximation from literature](#Apply-efficiency-approximation-from-literature)
	* [Add geodata](#Add-geodata)
* [Validity Checks](#Validity-Checks)
	* [Visual validity check](#Visual-validity-check)
		* [Sum of capacity by fuel type](#Sum-of-capacity-by-fuel-type)
		* [Capacities by plant status](#Capacities-by-plant-status)
		* [Power plant age](#Power-plant-age)
		* [Block size vs year of commissioning](#Block-size-vs-year-of-commissioning)
	* [Logical checks](#Logical-checks)
		* [Every power plant needs a capacity](#Every-power-plant-needs-a-capacity)
		* [Commissioning Dates](#Commissioning-Dates)
		* [Postcodes of BNetzA and UBA lists should match](#Postcodes-of-BNetzA-and-UBA-lists-should-match)
* [Documenting the data package (meta data)](#Documenting-the-data-package-%28meta-data%29)
* [Write the results to file](#Write-the-results-to-file)


# License

- This notebook is published under the LICENSENAME

# Prepare the environment

In [ ]:
import urllib.request
import csv
import pandas as pd
import numpy as np
import posixpath
import urllib.parse
import datetime  
import re
import os.path
import yaml  # http://pyyaml.org/, pip install pyyaml, conda install pyyaml
import json
import subprocess
from bokeh.charts import Scatter, show
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
import logging
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                              datefmt='%d %b %Y %H:%M:%S')
nb_root_logger.handlers[0].setFormatter(formatter)

#create download and output folder if they do not exist
if not os.path.exists('downloads/'): os.makedirs('downloads/')
if not os.path.exists('output/'): os.makedirs('output/')
if not os.path.exists('output/datapackage_powerplants_germany'): os.makedirs('output/datapackage_powerplants_germany')    

# Specify the source URLs:

In [ ]:
# BNetzA Power plant list
url_bnetza = 'http://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/Kraftwerksliste/Kraftwerksliste_CSV.csv?__blob=publicationFile&v=5'

# UBA Power plant list
url_uba = 'http://www.umweltbundesamt.de/sites/default/files/medien/376/dokumente/kraftwerke_in_deutschland_ab_100_megawatt_elektrischer_leistung_2015_09.xls'

# Define functions

This section defines functions used multiple times within this script

In [ ]:
def downloadandcache(url):
    """This function downloads a file into a folder called 
    downloads and returns the local filepath."""
    path = urllib.parse.urlsplit(url).path
    filename = posixpath.basename(path)
    now = datetime.datetime.now()
    datestring = ""
    datestring = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
    filepath = "downloads/"+datestring+"-"+filename
    
    #check if file exists, otherwise download it
    if os.path.exists(filepath) == False:
        print("Downloading file", filename)
        urllib.request.urlretrieve(url, filepath)
    else:
        print("Using local file from", filepath)
    filepath = './'+filepath
    return filepath


# Downloads

## Download the BNetzA power plant list

This section downloads the BNetzA power plant list and converts it to a pandas data frame

In [ ]:
bnetza_data_filepath=(downloadandcache(url_bnetza))
#print(bnetza_data_filepath)
plantlist=pd.read_csv(bnetza_data_filepath, 
               skiprows=9,
               sep=';',  # CSV field separator, default is ','
               thousands='.',  # Thousands separator, default is ','
               decimal=',',  # Decimal separator, default is '.')  
               encoding='cp1252')
plantlist.head()

## Download the Uba Plant list

In [ ]:
uba_data_filepath=(downloadandcache(url_uba))
plantlist_uba=pd.read_excel(uba_data_filepath,
                           skiprows=9
                           )
plantlist_uba.head()

# Translate contents

## Columns

A dictionary with the original column names to the new column names is created. This dictionary is used to translate the column names.

Original Name|Translation
-|-
Kraftwerksnummer Bundesnetzagentur|id
Unternehmen|company
Kraftwerksname|name
PLZ\n(Standort Kraftwerk)|postcode
Ort\n(Standort Kraftwerk)|city
Straße und Hausnummer (Standort Kraftwerk)|street
Bundesland|state
Blockname|block
Aufnahme der kommerziellen Stromerzeugung der derzeit in Betrieb befindlichen Erzeugungseinheit\n(Jahr)|commissioned
Kraftwerksstatus \n(in Betrieb/\nvorläufig stillgelegt/\nsaisonale Konservierung\nReservekraftwerk/\nSonderfall)|status
Energieträger|fuel_basis
Spezifizierung "Mehrere Energieträger" und "Sonstige Energieträger" - Hauptbrennstoff|fuel_multiple1
Spezifizierung "Mehrere Energieträger" - Zusatz- / Ersatzbrennstoffe|fuel_multiple2
Auswertung\nEnergieträger (Zuordnung zu einem Hauptenergieträger bei Mehreren Energieträgern)|fuel
Vergütungsfähig nach EEG\n(ja/nein)|eeg
Wärmeauskopplung (KWK)\n(ja/nein)|chp
Netto-Nennleistung (elektrische Wirkleistung) in MW|capacity
Bezeichnung Verknüpfungspunkt (Schaltanlage) mit dem Stromnetz der Allgemeinen Versorgung gemäß Netzbetreiber|network_node
Netz- oder Umspannebene des Anschlusses in kV|voltage
Name Stromnetzbetreiber|network_operator

In [ ]:
dict_columns = { 'Kraftwerksnummer Bundesnetzagentur':'id',
            'Unternehmen':'company',
            'Kraftwerksname':'name',
            'PLZ\n(Standort Kraftwerk)':'postcode',
            'Ort\n(Standort Kraftwerk)':'city',
            'Straße und Hausnummer (Standort Kraftwerk)':'street',
            'Bundesland':'state',
            'Blockname':'block',
            'Aufnahme der kommerziellen Stromerzeugung der derzeit in Betrieb befindlichen Erzeugungseinheit\n(Jahr)':'commissioned',
            'Kraftwerksstatus \n(in Betrieb/\nvorläufig stillgelegt/\nsaisonale Konservierung\nReservekraftwerk/\nSonderfall)':'status',
            'Energieträger':'fuel_basis',
            'Spezifizierung "Mehrere Energieträger" und "Sonstige Energieträger" - Hauptbrennstoff':'fuel_multiple1',
            'Spezifizierung "Mehrere Energieträger" - Zusatz- / Ersatzbrennstoffe':'fuel_multiple2',
            'Auswertung\nEnergieträger (Zuordnung zu einem Hauptenergieträger bei Mehreren Energieträgern)':'fuel',
            'Vergütungsfähig nach EEG\n(ja/nein)':'eeg',
            'Wärmeauskopplung (KWK)\n(ja/nein)':'chp',
            'Netto-Nennleistung (elektrische Wirkleistung) in MW':'capacity',
            'Bezeichnung Verknüpfungspunkt (Schaltanlage) mit dem Stromnetz der Allgemeinen Versorgung gemäß Netzbetreiber':'network_node',
            'Netz- oder Umspannebene des Anschlusses in kV':'voltage',
            'Name Stromnetzbetreiber':'network_operator',
            'Kraftwerksname / Standort': 'uba_name',
            'Betreiber ': 'uba_company',
            'Standort-PLZ': 'uba_postcode',
            'Kraftwerksstandort': 'uba_city',
            'Elektrische Bruttoleistung (MW)': 'uba_capacity',
            'Fernwärme-leistung (MW)': 'uba_chp_capacity',
            'Inbetriebnahme  (ggf. Ertüchtigung)': 'uba_commissioned',
            'Anlagenart': 'uba_technology',
            'Primärenergieträger': 'uba_fuel',
          }
plantlist.rename(columns=dict_columns, inplace=True)

# Check if all columns have been translated
for columnnames in plantlist.columns:
    #print(columnnames)
    if not columnnames in dict_columns.values():
        logger.error("Untranslated column: "+ columnnames)

## Fuel types

In [ ]:
dict_fuels = {'Steinkohle':'coal',
              'Erdgas':'gas',
              'Braunkohle':'lignite',
              'Kernenergie':'uranium',
              'Pumpspeicher':'pumped_storage',
              'Biomasse':'biomass',
              'Mineralölprodukte':'oil',
              'Laufwasser':'run_of_river',
              'Sonstige Energieträger\n(nicht erneuerbar) ':'other_non_renewable',
              'Abfall':'waste',
              'Speicherwasser (ohne Pumpspeicher)':'reservoir',
              'Unbekannter Energieträger\n(nicht erneuerbar)':'unknown_non_renewable',
              'Mehrere Energieträger\n(nicht erneuerbar)':'multiple_non_renewable',
              'Deponiegas':'gas_landfill',
              'Windenergie (Onshore-Anlage)':'wind_onshore',
              'Windenergie (Offshore-Anlage)':'wind_offshore',
              'Solare Strahlungsenergie':'solar',
              'Klärgas':'gas_sewage',
              'Geothermie':'geothermal',
              'Grubengas':'gas_mine'
                        }
plantlist["fuel"].replace(dict_fuels, inplace=True)
plantlist["fuel"].unique()

# Check if all fuels have been translated
for fuelnames in plantlist["fuel"].unique():
    #print(columnnames)
    if not fuelnames in dict_fuels.values():
        logger.error("Untranslated fuel: "+ fuelnames)


## Power plant status

In [ ]:
dict_plantstatus ={
'in Betrieb':'operating',
'vorläufig stillgelegt':'shutdown_temporal',
'Sonderfall':'special_case',
'saisonale Konservierung':'seasonal_conservation',
'Reservekraftwerk':'reserve',
'Endgültig Stillgelegt 2011':'shutdown_2011',
'Endgültig Stillgelegt 2012':'shutdown_2012',
'Endgültig Stillgelegt 2013':'shutdown_2013',
'Endgültig Stillgelegt 2014':'shutdown_2014',
'Endgültig Stillgelegt 2015':'shutdown_2015',
'Endgültig stillgelegt 2015':'shutdown_2015',
}
plantlist["status"].replace(dict_plantstatus, inplace=True)
plantlist["status"].unique()

# Check if all fuels have been translated
for statusnames in plantlist["status"].unique():
    #print(columnnames)
    if not statusnames in dict_plantstatus.values():
        logger.error("Untranslated plant status: "+ statusnames)

## CHP Capability

In [ ]:
dict_chpcapability ={
'Nein':'no',
'nein':'no',
'Ja':'yes',
'ja':'yes',    
}
plantlist["chp"].replace(dict_chpcapability, inplace=True)
plantlist["chp"].unique()

# Check if all fuels have been translated
for chpnames in plantlist["chp"].unique():
    if (not chpnames in dict_chpcapability.values()) & (str(chpnames) != "nan"):
        logger.error("Untranslated chp capability: " + str(chpnames))

## UBA Columns

Using the same dictionary as above

In [ ]:
dict_uba_columns = {
            'Kraftwerksname / Standort': 'uba_name',
            'Betreiber ': 'uba_company',
            'Standort-PLZ': 'uba_postcode',
            'Kraftwerksstandort': 'uba_city',
            'Elektrische Bruttoleistung (MW)': 'uba_capacity',
            'Fernwärme-leistung (MW)': 'uba_chp_capacity',
            'Inbetriebnahme  (ggf. Ertüchtigung)': 'uba_commissioned',
            'Anlagenart': 'uba_technology',
            'Primärenergieträger': 'uba_fuel',
            'Bundesland':'uba_state',
          }
plantlist_uba.rename(columns=dict_uba_columns, inplace=True)

# Check if all columns have been translated
for columnnames in plantlist_uba.columns:
    #print(columnnames)
    if not columnnames in dict_uba_columns.values():
        logger.error("Untranslated column: "+ columnnames)

## UBA Technologies

In [ ]:
# Translate UBA Technologies here

# Process data

## Set index to the BNetzA power plant ID

In [ ]:
# Set Index to Kraftwerksnummer_Bundesnetzagentur
plantlist['bnetza_id'] = plantlist['id']
plantlist = plantlist.set_index('id')
plantlist.head()

## Merge data from UBA List

In [ ]:
# UBA List import here
matchinglist=pd.read_csv('inputs/matching_bnetza_uba.csv', 
               skiprows=0,
               sep=';',  # CSV field separator, default is ','
               thousands='.',  # Thousands separator, default is ','
               decimal=',',  # Decimal separator, default is '.')  
               encoding='utf8')
matchinglist['ID UBA'] = matchinglist['ID UBA']-1
matchinglist.head()

### case 1-1

In [ ]:
#Check for cases:
#1-1 One BNetzA ID to one UBA-ID

match1t1 = matchinglist[(matchinglist.duplicated(subset=['ID UBA'], keep=False) == False) & (matchinglist.duplicated(subset=['ID BNetzA'], keep=False)== False)]
match1t1 = match1t1.set_index('ID BNetzA')
match1t1 = pd.merge(match1t1, plantlist_uba, left_on='ID UBA', right_index=True, how='left')
match1t1['uba_capacity_original'] = match1t1['uba_capacity']
match1t1['uba_chp_capacity_original'] = match1t1['uba_chp_capacity']
match1t1.head()

### case n-1

In [ ]:
#Match multiple BNetza IDs to One UBA ID
# Matching structure: 
# bnetza_id uba_id
# 1         1
# 2         1
# 3         1
# 4         2
# 5         2

# Get relevant entries from the matchinglist and merge the corresponding UBA Data to the list.
matchnt1= matchinglist[(matchinglist.duplicated(subset=['ID UBA'], keep=False) == True) & (matchinglist.duplicated(subset=['ID BNetzA'], keep=False)== False)]
matchnt1 = pd.merge(matchnt1, plantlist_uba, left_on='ID UBA', right_index=True, how='left')
matchnt1 = matchnt1.set_index('ID BNetzA')

# Import BNetzA Capacities and CHP criterion into matchnt1 dataframe
plantlist_capacities = pd.DataFrame(plantlist[['capacity','chp']])
plantlist_capacities=plantlist_capacities.rename(columns = {'capacity':'capacity_bnetza'})
plantlist_capacities=plantlist_capacities.rename(columns = {'chp':'chp_bnetza'})
#print(plantlist_capacities)
matchnt1 = pd.merge(matchnt1, plantlist_capacities, left_index=True, right_index=True, how='left')

# Get sum of BNetzA Capacitites for each UBA Index and merge into matchnt1 dataframe
plantlist_uba_capacitysum = pd.DataFrame(matchnt1.groupby('ID UBA').sum()['capacity_bnetza'])
plantlist_uba_capacitysum=plantlist_uba_capacitysum.rename(columns ={'capacity_bnetza':'capacity_bnetza_aggregate'})
#print(plantlist_uba_capacitysum)
matchnt1 = pd.merge(matchnt1, plantlist_uba_capacitysum, left_on='ID UBA', right_index=True, how='left')

# Scale UBA Capacities based BNEtza Data
matchnt1['uba_capacity_scaled'] = matchnt1['uba_capacity'] * matchnt1['capacity_bnetza']/matchnt1['capacity_bnetza_aggregate']

#determine sum of capacities with chp capability and add to matchnt1
plantlist_uba_chp_capacities = matchnt1[(matchnt1['chp_bnetza'] == 'yes')]
plantlist_uba_chp_capacitysum = pd.DataFrame(plantlist_uba_chp_capacities.groupby('ID UBA').sum()[['capacity_bnetza']])
plantlist_uba_chp_capacitysum = plantlist_uba_chp_capacitysum.rename(columns = {'capacity_bnetza':'capacity_bnetza_with_chp'})
plantlist_uba_chp_capacitysum
matchnt1 = pd.merge(matchnt1, plantlist_uba_chp_capacitysum, left_on='ID UBA', right_index=True, how='left')

matchnt1['uba_chp_capacity_scaled'] = matchnt1['uba_chp_capacity'] * matchnt1['capacity_bnetza']/matchnt1['capacity_bnetza_with_chp']

# Change column names for merge later on
matchnt1['uba_chp_capacity_original'] = matchnt1['uba_chp_capacity']
matchnt1['uba_chp_capacity'] = matchnt1['uba_chp_capacity_scaled']
matchnt1['uba_capacity_original'] = matchnt1['uba_capacity']
matchnt1['uba_capacity'] = matchnt1['uba_capacity_scaled']

# Drop columns not needed anymore
colsToDrop = ['capacity_bnetza', 'chp_bnetza','capacity_bnetza_with_chp', 'capacity_bnetza_aggregate', 'uba_chp_capacity_scaled', 'uba_capacity_scaled']
matchnt1 = matchnt1.drop(colsToDrop, axis=1)
matchnt1.head()

### case 1-n

In [ ]:
#1-n Case here
#The resulting DataFrame should be called "match1tn"
# Matching structure: 
# bnetza_id uba_id
# 1         1
# 1         2
# 1         3
# 2         4
# 2         5

# Get relevant entries from the matchinglist and merge the corresponding UBA Data to the list.
match1tn= matchinglist[(matchinglist.duplicated(subset=['ID BNetzA'], keep=False) == True) & (matchinglist.duplicated(subset=['ID UBA'], keep=False)== False)]
match1tn = pd.merge(match1tn, plantlist_uba, left_on='ID UBA', right_index=True, how='left')
match1tn = match1tn.set_index('ID BNetzA')
match1tn.head()

# Import BNetzA Capacities and CHP criterion into match1tn dataframe
plantlist_capacities = pd.DataFrame(plantlist[['capacity','chp']])
plantlist_capacities=plantlist_capacities.rename(columns = {'capacity':'capacity_bnetza'})
plantlist_capacities=plantlist_capacities.rename(columns = {'chp':'chp_bnetza'})
#plantlist_capacities.head()
#print(plantlist_capacities)
match1tn = pd.merge(match1tn, plantlist_capacities, left_index=True, right_index=True, how='left')
match1tn.index.names=['ID BNetzA']
match1tn.head()

# Get sum of UBA Capacitites for each BNetzA Index and merge into match1tn dataframe
plantlist_bnetza_capacitysum = pd.DataFrame(match1tn.groupby(match1tn.index).sum()['uba_capacity'])
plantlist_bnetza_capacitysum=plantlist_bnetza_capacitysum.rename(columns ={'uba_capacity':'uba_capacity_aggregate'})
#print(plantlist_uba_capacitysum)
match1tn = pd.merge(match1tn, plantlist_bnetza_capacitysum, left_index=True, right_index=True, how='left')

# Get sum of UBA CHP Capacities for each BNetzA Index and merge inot match1tn datafram
plantlist_bnetza_chp_capacitysum = pd.DataFrame(match1tn.groupby(match1tn.index).sum()['uba_chp_capacity'])
plantlist_bnetza_chp_capacitysum=plantlist_bnetza_chp_capacitysum.rename(columns ={'uba_chp_capacity':'uba_chp_capacity_aggregate'})
match1tn = pd.merge(match1tn, plantlist_bnetza_chp_capacitysum, left_index=True, right_index=True, how='left')

# Get UBA Technology for each BNetzA Index and merge into match1tn datafram
#plantlist_bnetza_chp_technology = pd.DataFrame(match1tn.groupby(match1tn.index).transform(lambda x: ', '.join(x))['uba_technology'])
#plantlist_bnetza_chp_technology=plantlist_bnetza_chp_technology.rename(columns ={'uba_technology':'uba_chp_technology_aggregate'})
#match1tn = pd.merge(match1tn, plantlist_bnetza_chp_technology, left_index=True, right_index=True, how='left')
match1tn['uba_technology_aggregate'] = pd.DataFrame(match1tn.groupby(match1tn.index).transform(lambda x: ', '.join(x))['uba_technology'])
match1tn['uba_name_aggregate'] = pd.DataFrame(match1tn.groupby(match1tn.index).transform(lambda x: ', '.join(x))['uba_name'])
#match1tn['uba_company_aggregate'] = pd.DataFrame(match1tn.groupby(match1tn.index).transform(lambda x: ', '.join(x))['uba_company'])

match1tn['uba_company_aggregate'] = pd.DataFrame(match1tn.groupby(match1tn.index)['uba_company'].agg(lambda x:x.value_counts().index[0]))
match1tn['uba_technology_aggregate'] = pd.DataFrame(match1tn.groupby(match1tn.index)['uba_technology'].agg(lambda x:x.value_counts().index[0]))
#test= match1tn.groupby(match1tn.index)['uba_company'].agg(lambda x:x.value_counts().index[0])

# Change column names for merge later on
match1tn['uba_chp_capacity_original'] = match1tn['uba_chp_capacity_aggregate']
match1tn['uba_capacity_original'] = match1tn['uba_capacity_aggregate']

# Drop columns not needed anymore
colsToDrop = ['capacity_bnetza', 'chp_bnetza','uba_chp_capacity_aggregate', 'uba_capacity_aggregate']
match1tn = match1tn.drop(colsToDrop, axis=1)

#!!! WARNING/TODO: Only fist entry will be taken and other entries will be skipped. CHECK FOR OTHER COLUMNS IS REQUIRED (e.g. Technology)

# Drop duplicate rows and keep first entry
#match1tn = match1tn.reset_index().drop_duplicates(subset='ID BNetzA', keep='first').set_index('ID BNetzA')

#plantlist_capacities.head()
#plantlist_bnetza_capacitysum.head()

match1tn.head()
#test.head()

### Merge into plantlist

In [ ]:
# Merge the UBA DataFrames

#Merge first two dataframes
plantlist_uba_for_merge = match1t1.append(matchnt1)

#Add third dataframe (Comment this in when the 1-n case is ready!)
plantlist_uba_for_merge = plantlist_uba_for_merge.append(match1tn)

#Merge plantlist_uba_for_merge into the plantlist
plantlist = pd.merge(plantlist, plantlist_uba_for_merge, left_index=True, right_index=True, how='left')

#plantlist.head()


In [ ]:
plantlist.loc[['BNA0073']]

## Delete fuels not in focus

In [ ]:
# Delete unwanted fuels
plantlist = plantlist[plantlist.fuel != 'solar']
plantlist = plantlist[plantlist.fuel != 'wind_onshore']
plantlist = plantlist[plantlist.fuel != 'wind_offshore']

# Delete placeholder values
plantlist = plantlist[plantlist.company != 'EEG-Anlagen < 10 MW']
plantlist = plantlist[plantlist.company != 'Nicht-EEG-Anlagen < 10 MW']

## Convert input colums to usable data types

In [ ]:
plantlist['capacity_float'] = pd.to_numeric(plantlist['capacity'],errors='coerce')
plantlist['commissioned_float'] = pd.to_numeric(plantlist['commissioned'],errors='coerce')
#plantlist.head()

## Add efficiency data

### Efficiencies from research

####Import data

In [ ]:
# Efficiencies
data_efficiencies_bnetza=pd.read_csv('inputs/input_efficiency_de.csv',
                                     sep=';',  # CSV field separator, default is ','
                                     decimal='.',  # Decimal separator, default is '.')  
                                     encoding='cp1252')
data_efficiencies_bnetza = data_efficiencies_bnetza.set_index('id')
data_efficiencies_bnetza['efficiency_net'] = pd.to_numeric(data_efficiencies_bnetza['efficiency_net'],errors='coerce')
data_efficiencies_bnetza = data_efficiencies_bnetza.dropna(subset=['efficiency_net'])
data_efficiencies_bnetza

plantlist = pd.merge(plantlist, data_efficiencies_bnetza, left_index=True, right_index=True, how='left')
plantlist.head()
plantlist['efficiency_net']


#### Plot efficiencies by year of commissioning

In [ ]:
plantlist_for_efficiency_analysis = plantlist
plantlist_for_efficiency_analysis = plantlist_for_efficiency_analysis.dropna(subset=['efficiency_net'])
fuel_for_plot = ['lignite', 'coal', 'oil', 'gas']
plantlist_for_efficiency_analysis = plantlist_for_efficiency_analysis[plantlist_for_efficiency_analysis.fuel.isin(fuel_for_plot)]
plot_efficiency_type = Scatter(plantlist_for_efficiency_analysis, 
                              notebook=True, 
                              x='commissioned_float', 
                              y='efficiency_net',
                              color='fuel', 
                              title='Efficiency vs commissioning year', 
                              xlabel='Year', 
                              ylabel='Efficiency',
                              legend="top_left",
                              height=700,
                              width=1000,
                             )
show(plot_efficiency_type)

#### Determine least-squares approximation based on researched data

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt

olslist = {}
for fuelnames in plantlist["fuel"].unique():
    plantlist_for_efficiency_analysis = plantlist[(plantlist.fuel==fuelnames) & (plantlist.efficiency_net.notnull()==True)]
    if len(plantlist_for_efficiency_analysis.index)>=4:
        efficiencyestimate = ols("efficiency_net  ~  commissioned_float + chp +uba_technology ", plantlist_for_efficiency_analysis).fit()
        #Other possible inputs: + chp + capacity_float + status + np.square(commissioned_float) + np.log(commissioned_float)
        #efficiencyestimate.summary()
        olslist[fuelnames]=efficiencyestimate
        #print(efficiencyestimate.params)
        print(efficiencyestimate.summary())
        #estimatedict = efficiencyestimate.params.to_dict()
        #plantlist['efficiency_estimated'] = estimatedict['Intercept']
        #for key in estimatedict:
        #    if key != 'Intercept':
        #        plantlist['efficiency_estimated'][plantlist.fuel==fuelnames] += estimatedict[key] * plantlist[key][plantlist.fuel==fuelnames]
        
        fig, ax = plt.subplots()
        fig = sm.graphics.plot_fit(efficiencyestimate, 'commissioned_float',  ax=ax)
        plt.ylabel("Efficiency")
        plt.xlabel("Commissioned")
        plt.title(fuelnames)
        #plt.plot(plantlist['commissioned_float'][plantlist.fuel==fuelnames], plantlist['efficiency_net'][plantlist.fuel==fuelnames], 'ro')
        #plt.plot(plantlist['commissioned_float'][plantlist.fuel==fuelnames], plantlist['efficiency_estimated'][plantlist.fuel==fuelnames], 'bo')
        plt.legend(['Data', 'Fitted model'], loc=2)
        #plt.axis([1900, 2020, 0.15, 1])
        plt.show()

#### Apply efficiency approximation from least squares approximation

### Efficiencies from literature
Jonas Egerer, Clemens Gerbaulet, Richard Ihlenburg, Friedrich Kunz, Benjamin Reinhard, Christian von Hirschhausen, Alexander Weber, Jens Weibezahn (2014): **Electricity Sector Data for Policy-Relevant Modeling: Data Documentation and Applications to the German and European Electricity Markets**. DIW Data Documentation 72, Berlin, Germany.

#### Import data

In [ ]:
data_efficiencies_literature=pd.read_csv('inputs/input_efficiency_literature_by_fuel_technology.csv',
                                     sep=',',  # CSV field separator, default is ','
                                     decimal='.',  # Decimal separator, default is '.')  
                                     encoding='utf8')
data_efficiencies_literature

#### Apply efficiency approximation from literature

## Add geodata

In [ ]:
data_plant_locations=pd.read_csv('inputs/input_plant_locations_de.csv',
                                     sep=';',  # CSV field separator, default is ','
                                     decimal='.',  # Decimal separator, default is '.')  
                                     encoding='cp1252')

data_plant_locations = data_plant_locations.set_index('id')
#data_plant_locations['googlemapslink'] = 'https://www.google.de/maps/@' + data_plant_locations['lat'].map(str) + ',' + data_plant_locations['lon'].map(str) + ',16.75z'
data_plant_locations['googlemapslink'] = 'http://maps.google.com/maps?q=loc:' + data_plant_locations['lat'].map(str) + ',' + data_plant_locations['lon'].map(str) + '&t=h'

data_plant_locations['lat'] = pd.to_numeric(data_plant_locations['lat'], errors='coerce')
data_plant_locations['lon'] = pd.to_numeric(data_plant_locations['lon'], errors='coerce')

plantlist = pd.merge(plantlist, data_plant_locations, left_index=True, right_index=True, how='left')
plantlist.head()


# Validity Checks

This section tests the results in various ways to find errors

## Visual validity check

### Sum of capacity by fuel type

In [ ]:
pivot_fuel_capacity = pd.pivot_table(
                        plantlist, 
                        values='capacity', 
                        index='fuel', 
                        aggfunc=[np.sum]
                        )
pivot_fuel_capacity.sort_values(by='sum', inplace=True, ascending=0)
#pivot_fuel_capacity
pivot_fuel_capacity_plot=pivot_fuel_capacity.plot(kind='bar', legend=False, figsize=(12, 6))
pivot_fuel_capacity_plot.set_ylabel("MW")
pivot_fuel_capacity_plot

### Capacities by plant status

In [ ]:
pivot_status_capacity = pd.pivot_table(
                        plantlist, 
                        values='capacity',
                        columns='status',
                        index='fuel', 
                        aggfunc=np.sum
                        )
pivot_status_capacity.sort_values(by='operating', inplace=True, ascending=0)
#pivot_status_capacity
pivot_status_capacity_plot=pivot_status_capacity.plot(kind='barh', stacked=True,legend=True, figsize=(12, 6))
pivot_status_capacity_plot.set_xlabel("MW")
pivot_status_capacity_plot

### Power plant age

In [ ]:
plantlist_filtered = plantlist#[plantlist.fuel=='coal']
pivot_age_capacity = pd.pivot_table(
                        plantlist_filtered, 
                        values='capacity',
                        columns='fuel',
                        index='commissioned', 
#                        index='fuel',
#                        columns='commissioned',     
                        aggfunc=np.sum,
                        dropna=True
                        )
#pivot_age_capacity
pivot_age_capacity_plot=pivot_age_capacity.plot(kind='bar', stacked=True,legend=True, figsize=(17, 10))
pivot_age_capacity_plot.set_ylabel("MW")
pivot_age_capacity_plot

### Block size vs year of commissioning

This chart is suitable to check outliers of commissioning years and block sizes. 
In theory, there should be no unexpected values, e.g. all commissioning years should be greater than 1900. 
Block sizes above 2000 MW are also unlikely.

In [ ]:
plantlist_for_plot = plantlist.copy(deep=True)
plantlist_for_plot['capacity_float'] = pd.to_numeric(plantlist_for_plot['capacity'], errors='coerce')
plantlist_for_plot['commissioned_float'] = pd.to_numeric(plantlist_for_plot['commissioned'], errors='coerce')

In [ ]:
plot_blocksize_year = Scatter(plantlist_for_plot, 
                              notebook=True, 
                              x='commissioned_float', 
                              y='capacity_float',
                              color='fuel', 
                              title='Block-Size vs Year of Commissioning', 
                              xlabel='Year', 
                              ylabel='MW',
                              legend="top_left",
                              height=500,
                              width=700,
                             )
show(plot_blocksize_year)

## Logical checks

### Every power plant needs a capacity

List all entries with zero capacity.

In [ ]:
plantlist[plantlist.capacity == 0]

### Commissioning Dates

In [ ]:
#Show all Plants with commisioning dates below 1900 
plantlist[plantlist['commissioned_float'] <=1900]

In [ ]:
#Show all Plants with invalid commisioning dates
plantlist[plantlist['commissioned_float'].isnull()]

### Postcodes of BNetzA and UBA lists should match

In [ ]:
# List all entries with diverging postcodes (if a postcode is given)
plantlist[(plantlist['uba_postcode'].notnull() == True) & (pd.to_numeric(plantlist.postcode, errors='coerce') != pd.to_numeric(plantlist.uba_postcode, errors='coerce'))]

# Documenting the data package (meta data)

We document the data packages meta data in the specific format JSON as proposed by the Open Knowledge Foundation. See the Frictionless Data project by OKFN (http://data.okfn.org/) and the Data Package specifications (http://dataprotocols.org/data-packages/) for more details.

In order to keep the notebook more readable, we first formulate the metadata in the human-readable YAML format using a multi-line string. We then parse the string into a Python dictionary and save that to disk as a JSON file.

In [ ]:
# Here we define meta data of the resulting data package.
# The meta data follows the specification at:
# http://dataprotocols.org/data-packages/

metadata = """

name: opsd-power-plants-germany
title: List of power plants in Germany.
description: This dataset contains an augmented and corrected power plant list based on the power plant list provided by the BNetzA.
version: 0.0.1
keywords: [power plants,germany]

resources:
    path: power_plants_germany.csv
    format: csv
    mediatype: text/csv
    schema:  # Schema according to: http://dataprotocols.org/json-table-schema/        
        fields:
            - name: id
              description: Power plant ID used in this list. Mostly based on the ID Provided in the BNetzA-list.
              type: string
            - name: company
              description: Company name
              type: string
            - name: name
              description: 
              type: 
              format:
            - name: postcode
              description: 
              type: 
              format:
            - name: city
              description: 
              type: 
              format:
            - name: street
              description: 
              type: 
              format:
            - name: state
              description: 
              type: 
              format:
            - name: block
              description: 
              type: 
              format:
            - name: commissioned
              description: 
              type: 
              format:
            - name: status
              description: 
              type: 
              format:
            - name: fuel_basis
              description: 
              type: 
              format:
            - name: fuel_multiple1
              description: 
              type: 
              format:
            - name: fuel_multiple2
              description: 
              type: 
              format:
            - name: fuel
              description: 
              type: 
              format:
            - name: eeg
              description: 
              type: 
              format:
            - name: chp
              description: 
              type: 
              format:
            - name: capacity
              description: 
              type: 
              format:
            - name: network_node
              description: 
              type: 
              format:
            - name: voltage
              description: 
              type: 
              format:
            - name: network_operator
              description: 
              type: 
              format:

licenses:
    - url: http://example.com/license/url/here
      name: License Name Here
      version: 1.0
      id: license-id-from-open

sources:
    - name: BNetzA Kraftwerksliste,
      web: http://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/Kraftwerksliste/kraftwerksliste-node.html
    - name: Umweltbundesamt Datenbank Kraftwerke in Deutschland,
      web: http://www.umweltbundesamt.de/dokument/datenbank-kraftwerke-in-deutschland

maintainers:
    - name: OPSD-Team,
      email: OPSD-Team-email,
      web: http://open-power-system-data.org/

views:
    # You can put hints here which kind of graphs or maps make sense to display your data. This makes the 
    # Data Package Viewer at http://data.okfn.org/tools/view automatically display visualazations of your data.
    # See http://data.okfn.org/doc/data-package#views for more details.    

# extend your datapackage.json with attributes that are not
# part of the data package spec
# you can add your own attributes to a datapackage.json, too

openpowersystemdata-enable-listing: True  # This is just an example we don't actually make use of yet.


"""

metadata = yaml.load(metadata)

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

# Write the results to file

Write the outputs

In [ ]:
output_path = 'output/datapackage_powerplants_germany/'
output_path2 = 'output/datapackage_powerplants_germany'

#Write the result to file
plantlist.to_csv(output_path+'power_plants_germany.csv', encoding='utf-8')

#Write the results to excel file
plantlist.to_excel(output_path+'power_plants_germany.xlsx', sheet_name='output')

#Write the information of the metadata
with open(os.path.join(output_path, 'datapackage.json'), 'w') as f:
    f.write(datapackage_json)

#Set this string to this notebook's filename!    
nb_filename = 'Power_Plants_DE.ipynb'

# Save a copy of the notebook to markdown, to serve as the package README file
subprocess.call(['ipython', 'nbconvert', '--to', 'markdown', nb_filename])
path_readme = os.path.join(output_path2, 'README.md')
try:
    os.remove(path_readme)
except Exception:
    pass
os.rename(nb_filename.replace('.ipynb', '.md'), path_readme)    
    